# Project 1 : 환율 예측

- DATA SET은 2018년1월부터 2020년 1월까지.(총 2년 1개월)
- SET01 ~ SET12는 DATA SET을 TRAIN SET 1년, TEST SET 1개월로 나누고 한달 간격으로 총 12개의 SET을 만듬(SLIDING WINDOW 방식)
- prophet과 arima는 고시 환율만을 사용하여 모델을 구성하였으며, dnn은 환율, 미국과 한국의 3개월 리보금리, kospi300, s&p500 활용
- 1,2,3번 전부 각각의 set을 통해 예측한 확률의 평균을 구성하였으며, 3번의 마지막 표만 1년을 예측하여 11개월을 평가한 것이다.

---------------

# 1. facebook prophet

- 페이스북에서 개발한 시계열 예측 알고리즘이다. ARIMA 모델처럼 이론적으로 구성된 모형이 아니라 지금까지의 추세를 활용하므로 단기 시계열 예측에 좋은 성능으로 알려져 있다. prophet모델을 통해 장기적인 예측은 할 수 없지만, 단기에 환율의 방향을 예측할 수 있다. 

- prophet 모델을 이용하여  5일간의 환율 예측치를 예측한 결과 RMSE가 큰 차이를 보였으며, 또한 실제가격과 예측가격의 상승,하락만을 비교하였을때의 정확도는 약 42%정도로 환율 예측에 사용할 수 없는 모델로 생각된다. 이를 보완하기 위해 ARIMA모델을 사용한다.

## < 코드 >

    import numpy as np
    import pandas as pd
    import seaborn as sns
    import matplotlib.pyplot as plt
    from fbprophet import Prophet
    from datetime import datetime, timedelta

    from sklearn.metrics import mean_squared_error
    from sklearn.preprocessing import MinMaxScaler

### data import

    data=pd.read_csv('data/USD_KRW.csv')
    data.columns=['ds','y_original']
    data=data.set_index('ds')

#### 1. data set 시작, 끝 날짜 지정

    start_date_train = ['2018-02-01','2018-03-01','2018-04-01','2018-05-01','2018-06-01','2018-07-01','2018-08-01','2018-09-01',
                        '2018-10-01','2018-11-01','2018-12-01','2019-01-01']
    end_date_train = ['2019-01-31','2019-02-28','2019-03-31','2019-04-30','2019-05-31','2019-06-30','2019-07-31','2019-08-31',
                      '2019-09-30','2019-10-31','2019-11-30','2019-12-31']
    start_date_test = ['2019-02-01','2019-03-01','2019-04-01','2019-05-01','2019-06-01','2019-07-01','2019-08-01','2019-09-01',
                       '2019-10-01','2019-11-01','2019-12-01','2020-01-01']
    end_date_test = ['2019-02-28','2019-03-31','2019-04-30','2019-05-31','2019-06-30','2019-07-31','2019-08-31','2019-09-30',
                     '2019-10-31','2019-11-30','2019-12-31','2020-01-31']

#### 2. holiday 설정

    holiday_list = [
           #2018 휴일
           '2018-01-01','2018-01-06','2018-01-07','2018-01-13','2018-01-14','2018-01-20','2018-01-21','2018-01-27',
           '2018-01-28','2018-02-03','2018-02-04','2018-02-10','2018-02-11','2018-02-15','2018-02-16','2018-02-17',
           '2018-02-18','2018-02-24','2018-02-25','2018-03-01','2018-03-03','2018-03-04','2018-03-10','2018-03-11',
           '2018-03-17','2018-03-18','2018-03-24','2018-03-25','2018-03-31','2018-04-01','2018-04-07','2018-04-08',
           '2018-04-14','2018-04-15','2018-04-21','2018-04-22','2018-04-28','2018-04-29','2018-05-05','2018-05-06',
           '2018-05-07','2018-05-12','2018-05-13','2018-05-19','2018-05-20','2018-05-22','2018-05-26','2018-05-27',
           '2018-06-02','2018-06-03','2018-06-06','2018-06-09','2018-06-10','2018-06-13','2018-06-16','2018-06-17',
           '2018-06-23','2018-06-24','2018-06-30','2018-07-01','2018-07-07','2018-07-08','2018-07-14','2018-07-15',
           '2018-07-21','2018-07-22','2018-07-28','2018-07-29','2018-08-04','2018-08-05','2018-08-12','2018-08-15',
           '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
           '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
           '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
           '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
           '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
           '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
           #2019 휴일
           '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
           '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
           '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
           '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
           '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
           '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
           '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
           '2019-06-23','2019-06-29','2019-06-30','2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
           '2019-07-21','2019-07-27','2019-07-28','2019-08-03','2019-08-04','2019-08-10','2019-08-11','2019-08-15',
           '2019-08-17','2019-08-18','2019-08-24','2019-08-25','2019-08-31','2019-09-01','2019-09-07','2019-09-08',
           '2019-09-12','2019-09-13','2019-09-14','2019-09-15','2019-09-21','2019-09-22','2019-09-28','2019-09-29',
           '2019-10-03','2019-10-05','2019-10-06','2019-10-09','2019-10-12','2019-10-13','2019-10-19','2019-10-20',
           '2019-10-26','2019-10-27','2019-11-02','2019-11-03','2019-11-09','2019-11-10','2019-11-16','2019-11-17',
           '2019-11-23','2019-11-24','2019-11-30','2019-12-01','2019-12-07','2019-12-08','2019-12-14','2019-12-15',
           '2019-12-21','2019-12-22','2019-12-25','2019-12-28','2019-12-29',
           #2020 휴일
           '2020-01-01','2020-01-04','2020-01-05','2020-01-11','2020-01-12','2020-01-18','2020-01-19','2020-01-24',
           '2020-01-25','2020-01-26','2020-01-27']

#### 3. train, test set 나누기위해 holiday 구분

    trh1 = ['2018-02-03','2018-02-04','2018-02-10','2018-02-11','2018-02-15','2018-02-16','2018-02-17',
           '2018-02-18','2018-02-24','2018-02-25','2018-03-01','2018-03-03','2018-03-04','2018-03-10','2018-03-11',
           '2018-03-17','2018-03-18','2018-03-24','2018-03-25','2018-03-31','2018-04-01','2018-04-07','2018-04-08',
           '2018-04-14','2018-04-15','2018-04-21','2018-04-22','2018-04-28','2018-04-29','2018-05-05','2018-05-06',
           '2018-05-07','2018-05-12','2018-05-13','2018-05-19','2018-05-20','2018-05-22','2018-05-26','2018-05-27',
           '2018-06-02','2018-06-03','2018-06-06','2018-06-09','2018-06-10','2018-06-13','2018-06-16','2018-06-17',
           '2018-06-23','2018-06-24','2018-06-30','2018-07-01','2018-07-07','2018-07-08','2018-07-14','2018-07-15',
           '2018-07-21','2018-07-22','2018-07-28','2018-07-29','2018-08-04','2018-08-05','2018-08-12','2018-08-15',
           '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
           '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
           '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
           '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
           '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
           '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30','2019-01-01','2019-01-05','2019-01-06',
           '2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26','2019-01-27']
    trh2 = ['2018-03-01','2018-03-03','2018-03-04','2018-03-10','2018-03-11',
           '2018-03-17','2018-03-18','2018-03-24','2018-03-25','2018-03-31','2018-04-01','2018-04-07','2018-04-08',
           '2018-04-14','2018-04-15','2018-04-21','2018-04-22','2018-04-28','2018-04-29','2018-05-05','2018-05-06',
           '2018-05-07','2018-05-12','2018-05-13','2018-05-19','2018-05-20','2018-05-22','2018-05-26','2018-05-27',
           '2018-06-02','2018-06-03','2018-06-06','2018-06-09','2018-06-10','2018-06-13','2018-06-16','2018-06-17',
           '2018-06-23','2018-06-24','2018-06-30','2018-07-01','2018-07-07','2018-07-08','2018-07-14','2018-07-15',
           '2018-07-21','2018-07-22','2018-07-28','2018-07-29','2018-08-04','2018-08-05','2018-08-12','2018-08-15',
           '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
           '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
           '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
           '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
           '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
           '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
           '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
           '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
           '2019-02-16','2019-02-17','2019-02-23','2019-02-24']
    trh3 = ['2018-04-01','2018-04-07','2018-04-08',
           '2018-04-14','2018-04-15','2018-04-21','2018-04-22','2018-04-28','2018-04-29','2018-05-05','2018-05-06',
           '2018-05-07','2018-05-12','2018-05-13','2018-05-19','2018-05-20','2018-05-22','2018-05-26','2018-05-27',
           '2018-06-02','2018-06-03','2018-06-06','2018-06-09','2018-06-10','2018-06-13','2018-06-16','2018-06-17',
           '2018-06-23','2018-06-24','2018-06-30','2018-07-01','2018-07-07','2018-07-08','2018-07-14','2018-07-15',
           '2018-07-21','2018-07-22','2018-07-28','2018-07-29','2018-08-04','2018-08-05','2018-08-12','2018-08-15',
           '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
           '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
           '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
           '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
           '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
           '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
           '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
           '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
           '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
           '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31'] 
    trh4 = ['2018-05-05','2018-05-06',
           '2018-05-07','2018-05-12','2018-05-13','2018-05-19','2018-05-20','2018-05-22','2018-05-26','2018-05-27',
           '2018-06-02','2018-06-03','2018-06-06','2018-06-09','2018-06-10','2018-06-13','2018-06-16','2018-06-17',
           '2018-06-23','2018-06-24','2018-06-30','2018-07-01','2018-07-07','2018-07-08','2018-07-14','2018-07-15',
           '2018-07-21','2018-07-22','2018-07-28','2018-07-29','2018-08-04','2018-08-05','2018-08-12','2018-08-15',
           '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
           '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
           '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
           '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
           '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
           '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
           '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
           '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
           '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
           '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
           '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28']
    trh5 = ['2018-06-02','2018-06-03','2018-06-06','2018-06-09','2018-06-10','2018-06-13','2018-06-16','2018-06-17',
           '2018-06-23','2018-06-24','2018-06-30','2018-07-01','2018-07-07','2018-07-08','2018-07-14','2018-07-15',
           '2018-07-21','2018-07-22','2018-07-28','2018-07-29','2018-08-04','2018-08-05','2018-08-12','2018-08-15',
           '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
           '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
           '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
           '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
           '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
           '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
           '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
           '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
           '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
           '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
           '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
           '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26'] 
    trh6 = ['2018-07-01','2018-07-07','2018-07-08','2018-07-14','2018-07-15',
           '2018-07-21','2018-07-22','2018-07-28','2018-07-29','2018-08-04','2018-08-05','2018-08-12','2018-08-15',
           '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
           '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
           '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
           '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
           '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
           '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
           '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
           '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
           '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
           '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
           '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
           '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
           '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
           '2019-06-23','2019-06-29','2019-06-30']
    trh7 = ['2018-08-04','2018-08-05','2018-08-12','2018-08-15',
           '2018-08-18','2018-08-19','2018-08-25','2018-08-26','2018-09-01','2018-09-02','2018-09-08','2018-09-09',
           '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
           '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
           '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
           '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
           '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
           '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
           '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
           '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
           '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
           '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
           '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
           '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
           '2019-06-23','2019-06-29','2019-06-30','2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
           '2019-07-21','2019-07-27','2019-07-28']
    trh8 = ['2018-09-01','2018-09-02','2018-09-08','2018-09-09',
           '2018-09-15','2018-09-16','2018-09-22','2018-09-23','2018-09-24','2018-09-25','2018-09-29','2018-09-30',
           '2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
           '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
           '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
           '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
           '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
           '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
           '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
           '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
           '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
           '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
           '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
           '2019-06-23','2019-06-29','2019-06-30','2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
           '2019-07-21','2019-07-27','2019-07-28','2019-08-03','2019-08-04','2019-08-10','2019-08-11','2019-08-15',
           '2019-08-17','2019-08-18','2019-08-24','2019-08-25','2019-08-31']
    trh9 = ['2018-10-03','2018-10-06','2018-10-07','2018-10-09','2018-10-13','2018-10-14','2018-10-20','2018-10-21',
           '2018-10-27','2018-10-28','2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
           '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
           '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
           '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
           '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
           '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
           '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
           '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
           '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
           '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
           '2019-06-23','2019-06-29','2019-06-30','2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
           '2019-07-21','2019-07-27','2019-07-28','2019-08-03','2019-08-04','2019-08-10','2019-08-11','2019-08-15',
           '2019-08-17','2019-08-18','2019-08-24','2019-08-25','2019-08-31','2019-09-01','2019-09-07','2019-09-08',
           '2019-09-12','2019-09-13','2019-09-14','2019-09-15','2019-09-21','2019-09-22','2019-09-28','2019-09-29']
    trh10 = ['2018-11-03','2018-11-04','2018-11-10','2018-11-11','2018-11-17','2018-11-18',
           '2018-11-24','2018-11-25','2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
           '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
           '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
           '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
           '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
           '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
           '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
           '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
           '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
           '2019-06-23','2019-06-29','2019-06-30','2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
           '2019-07-21','2019-07-27','2019-07-28','2019-08-03','2019-08-04','2019-08-10','2019-08-11','2019-08-15',
           '2019-08-17','2019-08-18','2019-08-24','2019-08-25','2019-08-31','2019-09-01','2019-09-07','2019-09-08',
           '2019-09-12','2019-09-13','2019-09-14','2019-09-15','2019-09-21','2019-09-22','2019-09-28','2019-09-29',
           '2019-10-03','2019-10-05','2019-10-06','2019-10-09','2019-10-12','2019-10-13','2019-10-19','2019-10-20',
           '2019-10-26','2019-10-27']
    trh11 = ['2018-12-01','2018-12-02','2018-12-08','2018-12-09','2018-12-15','2018-12-16',
           '2018-12-22','2018-12-23','2018-12-25','2018-12-29','2018-12-30',
           '2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
           '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
           '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
           '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
           '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
           '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
           '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
           '2019-06-23','2019-06-29','2019-06-30','2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
           '2019-07-21','2019-07-27','2019-07-28','2019-08-03','2019-08-04','2019-08-10','2019-08-11','2019-08-15',
           '2019-08-17','2019-08-18','2019-08-24','2019-08-25','2019-08-31','2019-09-01','2019-09-07','2019-09-08',
           '2019-09-12','2019-09-13','2019-09-14','2019-09-15','2019-09-21','2019-09-22','2019-09-28','2019-09-29',
           '2019-10-03','2019-10-05','2019-10-06','2019-10-09','2019-10-12','2019-10-13','2019-10-19','2019-10-20',
           '2019-10-26','2019-10-27','2019-11-02','2019-11-03','2019-11-09','2019-11-10','2019-11-16','2019-11-17',
           '2019-11-23','2019-11-24','2019-11-30']
    trh12 = ['2019-01-01','2019-01-05','2019-01-06','2019-01-12','2019-01-13','2019-01-19','2019-01-20','2019-01-26',
           '2019-01-27','2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
           '2019-02-16','2019-02-17','2019-02-23','2019-02-24','2019-03-01','2019-03-02','2019-03-03','2019-03-09',
           '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31','2019-04-06',
           '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28','2019-05-04',
           '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26',
           '2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
           '2019-06-23','2019-06-29','2019-06-30','2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
           '2019-07-21','2019-07-27','2019-07-28','2019-08-03','2019-08-04','2019-08-10','2019-08-11','2019-08-15',
           '2019-08-17','2019-08-18','2019-08-24','2019-08-25','2019-08-31','2019-09-01','2019-09-07','2019-09-08',
           '2019-09-12','2019-09-13','2019-09-14','2019-09-15','2019-09-21','2019-09-22','2019-09-28','2019-09-29',
           '2019-10-03','2019-10-05','2019-10-06','2019-10-09','2019-10-12','2019-10-13','2019-10-19','2019-10-20',
           '2019-10-26','2019-10-27','2019-11-02','2019-11-03','2019-11-09','2019-11-10','2019-11-16','2019-11-17',
           '2019-11-23','2019-11-24','2019-11-30','2019-12-01','2019-12-07','2019-12-08','2019-12-14','2019-12-15',
           '2019-12-21','2019-12-22','2019-12-25','2019-12-28','2019-12-29']
    th1 = ['2019-02-02','2019-02-03','2019-02-04','2019-02-05','2019-02-06','2019-02-09','2019-02-10',
           '2019-02-16','2019-02-17','2019-02-23','2019-02-24']
    th2 = ['2019-03-01','2019-03-02','2019-03-03','2019-03-09',
           '2019-03-10','2019-03-16','2019-03-17','2019-03-23','2019-03-24','2019-03-30','2019-03-31']
    th3 = ['2019-04-06',
           '2019-04-07','2019-04-13','2019-04-14','2019-04-20','2019-04-21','2019-04-27','2019-04-28']
    th4 = ['2019-05-04',
           '2019-05-05','2019-05-06','2019-05-11','2019-05-12','2019-05-18','2019-05-19','2019-05-25','2019-05-26']
    th5 = ['2019-06-01','2019-06-02','2019-06-06','2019-06-08','2019-06-09','2019-06-15','2019-06-16','2019-06-22',
           '2019-06-23','2019-06-29','2019-06-30']
    th6 = ['2019-07-06','2019-07-07','2019-07-13','2019-07-14','2019-07-20',
           '2019-07-21','2019-07-27','2019-07-28']
    th7 = ['2019-08-03','2019-08-04','2019-08-10','2019-08-11','2019-08-15',
           '2019-08-17','2019-08-18','2019-08-24','2019-08-25','2019-08-31']
    th8 = ['2019-09-01','2019-09-07','2019-09-08',
           '2019-09-12','2019-09-13','2019-09-14','2019-09-15','2019-09-21','2019-09-22','2019-09-28','2019-09-29']
    th9 = ['2019-10-03','2019-10-05','2019-10-06','2019-10-09','2019-10-12','2019-10-13','2019-10-19','2019-10-20',
           '2019-10-26','2019-10-27']
    th10 = ['2019-11-02','2019-11-03','2019-11-09','2019-11-10','2019-11-16','2019-11-17',
           '2019-11-23','2019-11-24','2019-11-30']
    th11 = ['2019-12-01','2019-12-07','2019-12-08','2019-12-14','2019-12-15',
           '2019-12-21','2019-12-22','2019-12-25','2019-12-28','2019-12-29']
    th12 = ['2020-01-01','2020-01-04','2020-01-05','2020-01-11','2020-01-12','2020-01-18','2020-01-19','2020-01-24',
           '2020-01-25','2020-01-26','2020-01-27']

    train_holiday=[trh1,trh2,trh3,trh4,trh5,trh6,trh7,trh8,trh9,trh10,trh11,trh12]
    test_holiday=[th1,th2,th3,th4,th5,th6,th7,th8,th9,th10,th11,th12]

#### 3. 모델 구성

    for i in range(12):  # 총 12개의 train, test set

        df_train=data.loc[start_date_train[i] : end_date_train[i], : ]
        df_test=data.loc[start_date_test[i]: end_date_test[i], :]
        df_train=df_train.reset_index()
        df_test=df_test.reset_index()

        # train scale 조정

        y_original=np.array(df_train['y_original'])
        sc=MinMaxScaler((0.05,0.95))
        y=y_original.reshape(-1,1)
        y = sc.fit_transform(y)
        y_scale_train =pd.DataFrame(y)
        y_scale_train.columns=['y']

        df_train=pd.merge(df_train, y_scale_train["y"], how='outer', left_index=True, right_index=True)
        
        # test scale 조정

        y_scale_mean=df_train['y_original'].mean()
        y_scale_std=df_train['y_original'].std()

        df_test['y']=(df_test['y_original']-y_scale_mean)/y_scale_std

        df_train=df_train.reset_index()
        df_test=df_test.reset_index()

        # 예측값의 상한과 하한을 제어
        
        df_train['cap'] = 1.0
        df_train['floor'] = 0.0

        # holiday이지만 주말과 공휴일을 다 삭제 
        
        holiday = pd.DataFrame({'holiday': 'holiday', 'ds': pd.to_datetime(train_holiday[i]), 
                'lower_window': 0, 'upper_window': 0})
        m = Prophet(growth='logistic', holidays=holiday,yearly_seasonality=True, changepoint_range=0.9,
                   interval_width=0.95,daily_seasonality=True) # 여기서 파라미터 추가, 삭제 
        m.fit(df_train)

        # 미래 Dataframe 생성
        future = m.make_future_dataframe(periods=31)
        future.head()

        future=future.set_index('ds')
        future_train=future.loc[start_date_train[i] : end_date_train[i], : ]
        future_test=future.loc[start_date_test[i]: end_date_test[i], :]

        future_train=future_train.reset_index()
        future_test=future_test.reset_index()


        start_date = datetime.strptime(start_date_test[i], '%Y-%m-%d') #시작 날짜
        end_date = datetime.strptime(end_date_test[i], '%Y-%m-%d') #끝 날짜

        test_total_date = [] 

        while start_date.strftime('%Y-%m-%d') != end_date.strftime('%Y-%m-%d'):
            test_total_date.append(start_date.strftime('%Y-%m-%d')) 
            start_date += timedelta(days=1)


        test_holiday_date=test_holiday[i]    #마찬가지로 주말, 공휴일 제거 추후 추가

        final_date = list(set(test_total_date) - set(test_holiday_date))
        future_test=future_test[future_test['ds'].isin(final_date)]
        future=pd.concat([future_train,future_test],ignore_index=True)

        future['cap'] = 1.0
        future['floor'] = 0.0

        # 예측하기

        forecast = m.predict(future)  
        #forecast.iloc[-(len(start_date_test[i]) + len(end_date_test[i])):, :].yhat.plot()
        forecast=forecast[['ds', 'yhat', 'yhat_lower', 'yhat_upper']]

        # 옵션 1. 예측한 상한, 하한 범위와 추세 방향을 그래프로 시각화
        m.plot(forecast)
        
        # 옵션 2. 예측한 부분을 실제값과 그래프로 시각화
        pred=forecast[-len(df_test):].reset_index()

        plt.plot(pred['yhat'])
        plt.plot(df_test['y'])
        plt.show()

### 결과

- 5일간 상승, 하락 예상 -> 정확도 41.6%  

| set01 | set02 | set03 | set04 |  set05 | set06 | set07 | set08 | set09 | set10 | set11 | set12 |   
| -------- | ------- | -------- | -------- |  -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- |
| 60% | 0% | 100% | 0% | 20% | 40% | 20% | 40% | 20% | 60% | 100% | 40% |

- 다음날 상승, 하락 예상 -> 정확도 25%

| set01 | set02 | set03 | set04 |  set05 | set06 | set07 | set08 | set09 | set10 | set11 | set12 |   
| -------- | ------- | -------- | -------- |  -------- | -------- | -------- | -------- | -------- | -------- | -------- | -------- |
| X | X | O | X | X | X | X | X | X | O | O | X |

--------------------

# 2. ARIMA 

- ARIMA모형은 시계열 데이터 기반 분석 기법으로 과거 지식이나 경험을 바탕으로 움직이고 있음을 기초로 한다. ARIMA모형은 과거의 관측 값과 오차를 현재의 시계열 값을 설명하는 ARMA모델에서 다소 비안정적인 시계열의 특징에 적용이 가능하도록 발전한 모델이다.

- ARIMA를 이용하여 10개의 모델을 이용하여 예측한 결과 RMSE가 약 8~9원 정도가 나왔다. 모델을 구축하고 5일동안의 실제가격과 예측가격의 상승,하락만을 비교하였을때의 정확도는 대부분의 모델이 40% 정도의 성능을 보여주었다. prophet의 모델과 비교하였을때와 큰 차이는 없었지만, RMSE가 현저하게 줄었음을 볼 수 있다. 

## < 코드 > 
set01을 이용

    import pandas as pd
    import matplotlib.pyplot as plt

    from statsmodels.tsa.arima_model import ARIMA
    from statsmodels.graphics.tsaplots import plot_acf, plot_pacf
    from sklearn.metrics import mean_squared_error

### data import 

    data = pd.read_csv('exchange_data/set01.csv', header=0, squeeze=True)
    data = data.set_index('Date')
    
    # 2018년 2월~2019년1월 train set/ 2019년 2월 test set
    df_train=data.loc['2018-02-01': '2019-01-31' ]
    df_test=data.loc['2019-02-01':]

### model 구현 (최종 이용한 모델)

    #model1	

    model_1 = ARIMA(df_train, order=(1,1,1))
    model_1_fit = model_1.fit(trend='nc',full_output=True, disp=1)
    print(model_1_fit.summary())

    #model2	

    model_2 = ARIMA(df_train, order=(1,1,1))
    model_2_fit = model_2.fit(trend='c',full_output=True, disp=1)
    print(model_2_fit.summary())

    #model3	

    model_3 = ARIMA(df_train, order=(2,1,1))
    model_3_fit = model_3.fit(trend='c',full_output=True, disp=1)
    print(model_3_fit.summary())

    #model4	

    model_4 = ARIMA(df_train, order=(2,1,1))
    model_4_fit = model_4.fit(trend='nc',full_output=True, disp=1)
    print(model_4_fit.summary())

    #model18	

    model_18 = ARIMA(df_train, order=(5,1,2))
    model_18_fit = model_18.fit(trend='nc',full_output=True, disp=1)
    print(model_18_fit.summary())

    #model19	

    model_19 = ARIMA(df_train, order=(5,1,1))
    model_19_fit = model_19.fit(trend='nc',full_output=True, disp=1)
    print(model_19_fit.summary())

    #model20	

    model_20 = ARIMA(df_train, order=(5,1,1))
    model_20_fit = model_20.fit(trend='c',full_output=True, disp=1)
    print(model_20_fit.summary())

    #model25	

    model_25 = ARIMA(df_train, order=(5,0,0))
    model_25_fit = model_25.fit(trend='c',full_output=True, disp=1)
    print(model_25_fit.summary())

    #model26

    model_26 = ARIMA(df_train, order=(0,1,5))
    model_26_fit = model_26.fit(trend='nc',full_output=True, disp=1)
    print(model_26_fit.summary())

    #model27

    model_27 = ARIMA(df_train, order=(0,1,5))
    model_27_fit = model_27.fit(trend='c',full_output=True, disp=1)
    print(model_27_fit.summary())

    #model28

    model_28 = ARIMA(df_train, order=(0,1,4))
    model_28_fit = model_28.fit(trend='nc',full_output=True, disp=1)
    print(model_28_fit.summary())

    #model29

    model_29 = ARIMA(df_train, order=(0,1,4))
    model_29_fit = model_29.fit(trend='c',full_output=True, disp=1)
    print(model_29_fit.summary())

### 예측후 평가

    Models = [model_1_fit, model_2_fit, model_3_fit, model_4_fit, model_18_fit, model_19_fit, model_20_fit, 
              model_25_fit, model_26_fit, model_27_fit, model_28_fit, model_29_fit]

    df_test=df_test[:5]
    actual_price=df_test['Rate'].tolist()[:5]
    RMSE_total=[]
    pred_price=[]

    for model in Models:
        pred = model.forecast(steps=5)
        pred_price_0=pred[0][:5]
        pred_price.extend(pred_price_0)

        plt.plot(pred_price_0)
        plt.plot(df_test)
        plt.show()

        RMSE = mean_squared_error(pred_price_0, actual_price)**0.5    
        RMSE_total.append(RMSE)

- prophet의 성능이 크게 좋지 않아서 추세만 확인하기 위해 arima모델을 사용함.    
- 모델 자체의 구현이 간단한만큼 다양한 경우의 수를 생각하여 31개의 모델을 활용함.  
- (0,0,0)~(5,1,5) 사이의 수를 활용하여 12개의 set에 모두 사용할수 있는 set은 12개.  
- 이를 활용하여 실제값과 예측값이 다음날 맞을 확률, 5일 동안 추세가 맞을 확률을 구해봄.  
- prophet보다는 좋은 성능을 보이지만, 여전히 50%를 넘지 못하는 한계가 존재한다.

- 5일간 상승, 하락 예상 -> 12개의 모델 평균  
  
| model1 | model2 | model3 | model4 |  model18 | model19 | model20 | model25 | model26 | model27 | model28 | model29 |     
| ---------- | :---------:| :----------: | ----------: |  ----------: | ----------: | ----------: | ----------: | ----------: | ----------: | ----------: | ----------: |  
| 36.67% | 36.67% | 36.67% |	33.33% |	38.33% |	40.00% |	35.00% |	40.00% |	36.67% |	33.33% |	36.67% |	35.00% |   
  
- 5일간 평균 RMSE  
  
| model1 | model2 | model3 | model4 |  model18 | model19 | model20 | model25 | model26 | model27 | model28 | model29 |   
| ---------- | :---------:| :----------: | ----------: |  ----------: | ----------: | ----------: | ----------: | ----------: | ----------: | ----------: | ----------: |
| 8.345  |  8.361 | 8.157  |  8.241 | 8.696  |  8.121 |  8.097 | 8.504  | 8.266  | 8.262  | 8.278  | 8.283  | 
  
- 다음날 상승, 하락 예상 -> 정확도 평균
  
| model1 | model2 | model3 | model4 |  model18 | model19 | model20 | model25 | model26 | model27 | model28 | model29 |   
| ---------- | :---------:| :----------: | ----------: |  ----------: | ----------: | ----------: | ----------: | ----------: | ----------: | ----------: | ----------: |
| 42.67% | 42.67% | 50% | 33.33% | 42.67% | 42.67% | 50% | 50% | 42.67% | 33.33% | 50% | 33.33% |

-----------------

# 3. Dnn 모델

- DNN은 심층 신경망으로 입력층과 출력층 사이에 여러개의 은닉층으로 이뤄진 인공 신경망이다. dnn모델의 장점은 연속형, 범주형 변수를 모두 분석이 가능하며, 다른 머신러닝 기법에 비해 성능이 우수한 경우가 많다. 따라서 분류 및 수치예측을 위해 다양한 분야에서 dnn모델이 사용되어지고 있다.

- dnn을 이용하여 다음날 환율의 가격과 상승, 하락을 예측하였다. 은닉층과 파라미터를 수정하며 다양하게 모델을 구성해 본 결과 RMSE는 10-11원 정도가 나왔다. 또한, 5일의 정확도는 80%로 나왔으며 총 11개월동안의 평균 정확도는 57~62% 정도의 성능을 보여주었다. 

## <  기본 코드 > 

    import pandas as pd
    import matplotlib.pyplot as plt
    import numpy as np

    from keras.models import Sequential
    from keras.layers import Dense, LSTM ,Dropout
    from keras.callbacks import EarlyStopping
    from keras import layers

    from sklearn.metrics import mean_squared_error

    데이터 전처리
    사용할 데이터 : 환율, 6개월 한국 미국 libor 금리, S&P 500, KOSPI200
    
    # 1. 환율
    rate=pd.read_csv('data/set01.csv')
    rate.columns=['date', 'rate']
    rate=rate.set_index('date')
    rate_train=rate.loc['2018-02-01': '2019-02-01' ]
    rate_test=rate.loc['2019-02-01':'2019-03-01']
    
    # 2. 한국 libor
    kolib=pd.read_csv('data/KORIBOR.csv')
    kolib=kolib[['date', '6month']]
    kolib.columns=['date','krlibor']
    kolib=kolib.set_index('date')
    kolib_train=kolib.loc['2018-02-01': '2019-01-31' ]
    kolib_test=kolib.loc['2019-02-01':'2019-03-01']
    
    # 3. 미국 libor

    uslib=pd.read_csv('data/libor.csv')
    uslib.columns=['date','uslibor']
    uslib=uslib.set_index('date')
    uslib_train=uslib.loc['2018-02-01': '2019-01-31' ]
    uslib_test=uslib.loc['2019-02-01':'2019-02-31']

    # 데이터에 일부분에 . 이 존재함. 
    # 국가 공휴일은 삭제하고 필요한 값은 평균을 활용
    del uslib_train['uslibor']['2018-12-25']
    del uslib_train['uslibor']['2019-01-01']
    uslib_train['uslibor'].loc['2018-03-30'] = (float(uslib_train['uslibor'].loc['2018-03-29']) * 2/3 
                                                + float(uslib_train['uslibor'].loc['2018-04-03']) * 1/3)

    uslib_train['uslibor'].loc['2018-04-02'] = (float(uslib_train['uslibor'].loc['2018-03-29']) * 1/3 
                                                + float(uslib_train['uslibor'].loc['2018-04-03']) * 2/3)

    uslib_train['uslibor'].loc['2018-05-07'] = (float(uslib_train['uslibor'].loc['2018-05-04']) * 1/2 
                                                + float(uslib_train['uslibor'].loc['2018-05-08']) * 1/2)

    uslib_train['uslibor'].loc['2018-05-28'] = (float(uslib_train['uslibor'].loc['2018-05-25']) * 1/2 
                                                + float(uslib_train['uslibor'].loc['2018-05-29']) * 1/2)

    uslib_train['uslibor'].loc['2018-08-27'] = (float(uslib_train['uslibor'].loc['2018-08-24']) * 1/2 
                                                + float(uslib_train['uslibor'].loc['2018-08-28']) * 1/2)

    uslib_train['uslibor'].loc['2018-12-26'] = (float(uslib_train['uslibor'].loc['2018-12-24']) * 2/3 
                                                + float(uslib_train['uslibor'].loc['2018-12-27']) * 1/3)
    # 값이 문자형으로 되어있어서 실수형으로 변경
    uslib_train['uslibor']=uslib_train['uslibor'].apply(pd.to_numeric)
    uslib_test['uslibor']=uslib_test['uslibor'].apply(pd.to_numeric)  
    
    # 4.s&p 500
    snp500=pd.read_csv('data/S&P500.csv')
    snp500 = snp500[::-1]
    snp500=snp500[['날짜','종가']]
    snp500.columns=['date','snp500']


    snp500_date=[]
    for i in snp500['date']:
        snp500_date.append(i.replace('년 ','-').replace('월 ','-').replace('일',''))

    snp500_price=[]
    for i in snp500['snp500']:
        i=i.replace(',','')
        snp500_price.append(np.float(i))

    snp500['date']=snp500_date
    snp500['snp500']=snp500_price
    snp500=snp500.set_index('date')

    snp500_train = snp500.loc['2018-02-01': '2019-01-31' ]
    snp500_test = snp500.loc['2018-02-01': '2019-02-31' ]
    
    # 5. kospi200
    kospi200=pd.read_csv('data/KOSPI200.csv')
    kospi200=kospi200[['날짜','종가']]
    kospi200.columns=['date','kospi200']
    kospi200 = kospi200[::-1]

    kospi_date=[]
    for i in kospi200['date']:
        kospi_date.append(i.replace('년 ','-').replace('월 ','-').replace('일',''))

    kospi200['date']=kospi_date
    kospi200=kospi200.set_index('date')


    kospi200_train = kospi200.loc['2018-02-01': '2019-01-31' ]
    kospi200_test=kospi200.loc['2019-02-01':'2019-03-01']

### 정리한 데이터를 활용하여 같은 날로 묶어서 하나의 dataframe 생성 후 정리

    data_train=pd.concat([rate_train,kolib_train,uslib_train,snp500_train,kospi200_train], axis=1,  join = 'inner', sort=False)
    data_test=pd.concat([rate_test,kolib_test,uslib_test,snp500_test,kospi200_test], axis=1,  join = 'inner', sort=False)

    data_train['y']=data_train['rate'].shift(-1)
    data_test['y']=data_test['rate'].shift(-1)

    #Nan 값을 계속 찾아서 업데이트 할 예정
    data_train['y'][-1]=rate_test.iloc[0]
    data_train=data_train[['krlibor','uslibor','snp500','kospi200','y']]

    data_test['y'][-1]=rate_test.iloc[0]
    data_test=data_test[['krlibor','uslibor','snp500','kospi200','y']]
    data_test=data_test[:-1]

    train_input = data_train[['krlibor','uslibor','snp500','kospi200']]
    train_target = data_train['y']
    test_input = data_test[['krlibor','uslibor','snp500','kospi200']]
    test_target = data_test['y']

### scale맞추기

    mean = train_input.mean(axis=0)
    std = train_input.std(axis=0)
    train_input -= mean
    train_input /= std
    test_input -= mean
    test_input /= std
    In [22]:
    train_input=np.array(train_input)
    train_input=train_input.reshape(train_input.shape[0],4)

### 모델 구현하기 (DNN)

    early_stopping = EarlyStopping(patience=50)

    model = Sequential()

    model.add(layers.Dense(units=1024, input_shape=(4,)))
    model.add(Dropout(0.1))

    model.add(layers.Dense(units=128, input_shape=(4,)))
    model.add(Dropout(0.1))

    model.add(Dense(units=1))

    model.compile(optimizer='tanh', loss='mean_squared_error')
    history = model.fit(train_input, train_target, epochs=1000, batch_size=32, validation_split=0.3, callbacks=[early_stopping])

### loss값 그래프로 표현

    y_vloss = history.history['val_loss']
    y_loss = history.history['loss']

    x_len = np.arange(len(y_loss))

    plt.figure(figsize=(20,5))
    plt.plot(x_len, y_vloss, marker='.', c='red', label="Validation-set Loss")
    plt.plot(x_len, y_loss, marker='.', c='blue', label="Train-set Loss")
    plt.legend(loc='upper right')
    plt.grid()
    plt.xlabel('epoch')
    plt.ylabel('loss')
    plt.show()

### 예측하고 평가

    predicted = model.predict(test_input)
    actual=data_test['y']

    RMSE = mean_squared_error(actual, predicted)**0.5    
    RMSE

### 실제값 예측값 dataframe 만들기

    predicted = sum(predicted.tolist(), [])
    actual=actual.tolist()

    final=pd.DataFrame([predicted, actual]).T
    final.columns=['pred','actual']
    final=final.set_index(data_test.index)

### 상승, 하락 표시

    #train set의 마지막 데이터
    lastday=rate_train[-1:]['rate']
    In [ ]:
    actual_updown=[]
    pred_updown=[]
    
    if final['actual'][0] > lastday[0]:
        actual_updown.append('up')
    else :
        actual_updown.appedn('down')

    if final['pred'][0] > lastday[0]:
        pred_updown.append('up')
    else :
        pred_updown.append('down')
        
    for i in range(len(final)-1):
    
        if final['actual'][i+1] > final['actual'][i]:
            actual_updown.append('up')
        else :
            actual_updown.append('down')
        
    for i in range(len(final)-1):

        if final['pred'][i+1] > final['actual'][i]:
            pred_updown.append('up')
        else :
            pred_updown.append('down')
            
    final['pred_updown']=pred_updown
    final['actual_updown']=actual_updown

    # 한달간 예측
    print('한달간 상승,하락 정확도 : ',round(sum(final['pred_updown'] == final['actual_updown'])/len(final)*100,2), '%')

    # 5일만 예측
    final_5=final[:5]
    print('5일간 상승,하락 정확도 : ',round(sum(final_5['pred_updown'] == final_5['actual_updown'])/len(final_5)*100,2), '%')

### 그림으로 표현

    plt.plot(predicted, label='pred')
    plt.plot(actual, label='actual')
    plt.legend(loc='best')
    plt.show()

## 결과 분석

set01만 활용하여 5일간 정확률 80%, RMSE=4.6으로 전체 11개(set01~set11)의 셋을 돌려봄.    
- RMSE  
    
| set01 | set02 | set03 | set04 |  set05 | set06 | set07 | set08 | set09 | set10 | set11 |    
| ---------- | :---------:| :----------: | ----------: |  ----------: | ----------: | ----------: | ----------: | ----------: | ----------: | ----------: |
| 3.44 | 8.80 | 22.01 | 38.40 | 16.21 | 63.51 | 49.08 | 29.64 | 48.24 | 24.62 | 6.39 |  

- 상승, 하락 예상(test set전체(한달), 5일, 다음날) -> 각각 확률 : 25%, 34.54%, 46.67%  
    
| 기준 | set01 | set02 | set03 | set04 |  set05 | set06 | set07 | set08 | set09 | set10 | set11 |     
| ---------- | :---------:| :----------: | ----------: |  ----------: | ----------: | ----------: | ----------: | ----------: | ----------: | ----------: | ----------: |
| 1일 | X | O | X | X | X | X | X | X | O | O | X |    
| 5일 | 60.0  | 100.0  | 40.0  | 0.0  | 80.0  | 20.0  | 0.0  | 0.0 | 20.0  | 20.0  | 40.0  |  
| 한달 | 40.0  | 78.95  | 30.0  | 15.79  | 77.78  | 38.1  | 35.0  | 47.06  | 35.0  | 52.63  | 63.16  |  

파라미터를 수정하면서 모델 구현(위의 코드의 일부 변경)
  
|모델	|	1층(유닛,활성함수,dropout,규제)	|	2층(유닛,활성함수,dropout,규제)|	3층(유닛,활성함수,dropout,규제)|	
| ---------- | :---------:| :----------: | ----------: |
|1번 	| 	1024, 0.1	|	128, 0.1 	|X|			
|2번 	| 	1024 relu, 0.1	|1024 relu, 0.1  	|X	|	
|3번 	| 	1024 relu, 0.2	|1024 relu, 0.1		|X	|
|4번 	| 	1024, 0.1	|	1024, 0.2		|1024,0.1 	|	
|5번 	| 	2048,0.2	|	1024, 0.2		|512,0.2 	 	|
|6번 |		512,0.2	|	512, 0.2	|	256,0.2  |
|7번 	|	1024,0.1	|	512, 0.1	|	256,0.1 |
|8번 |		1024,512	|	256, 0.1 	 	|X		|
|9번 	|	1024 softmax|	512		256,0.1  |X		|
|10번 |	1024,0.2	|	512, 0.2	|	256,0.2, l1=0.01  |
|11번 |	1024,0.1	|	512, 0.1	|	245,0.1, l2=0.01  |
|12번 |	1024, 0.1	|	512, 0.1, l2=0.01 |X			|
|13번 |	1024, 0.1	|	512, 0.1, l1=0.01	 	|X	|
|14번 |	1024,0.1	|	1024, 0.1  			|X|

| 기준 | model1 | model2 | model3 | model4 |  model5 | model6 | model7 | model8 | model9 | model10 | model11 | model12 | model13 | model14 |     
| ---------- | :---------:| :----------: | ----------: |  ----------: | ----------: | ----------: | ----------: | ----------: | ----------: | ----------: | ----------: | ----------: | ----------: | ----------: |    
| RMSE | 30.36 | 160.09 | 166.45 | 26.99 | 25.58 | 51.28 | 32.49 | 35.70 | 34.51 | 30.75 | 39.12 | 32.39 | 31.64 | 36.16 |    
| 5일 | 36.36 | 45.45 | 27.27 | 40 | 50.09 | 45.45 | 40 | 45.45 | 47.27 | 41.81 | 29.09 | 36.36 | 34.54 | 36.36 |  
| 한달 | 46.10 | 45.34 | 45.05 | 45.52 | 48.71 | 47.39 | 44.92 | 50.14 | 47.94 | 48.41 | 44.64 | 44.79 | 46.93 | 44.89 |  

#### 예측 결과

- 은닉층을 1층만 쌓거나, 활성함수를 변경하면 오히려 값이 안좋게 나오는 경향이 있음. 
- 이를 통해 2층~3층 정도의 은닉층과 규제를 적절히 추가하는 법이 전체적으로 좋아지는 경향이 있음.
- 업다운의 정확도 보다는 RMSE를 토대로 온라인 학습으로 그래프를 그려서 시각가 좋아 보임.

- 성능이 괜찮았던 모델들로 1년 train셋, 11개월 test셋으로 하여 모델을 돌려봄.

| 모델 |	RMSE |	전체 |	100일 |	50일	| 20일 |	10일 |	5일 |
| ---------- | :---------:| :----------: | ----------: | ---------- | :---------:| :----------: | ----------: |
|1-1번 |	14.36	|52.07%|	50%|	58%	|70%|	70%|	80%|
|1-2번	| 11.08	|58.99%	|56%	|66%	|65%|	60%	|80%|
|1-3번	|11.03|	59.99%|	54%|	62%|	70%|	60%|	80%|
|4-1번|	33.23	|45.16%|	39%|	38%|	35%|	40%|	20%|
|5-1번|	23.12	|51.15%	|51%|	62%	|70%|	60%	|60%|
|8-1번	|10.74	|57.6%|	53%	|64%|	75%	|70%|	80%|
|8-2번|	10.69	|57.6%|	50%|	60%	|60%	|60%|	80%|
|8-3번|	10.8	|61.29%	|56%|	66%	|65%|	60%	|80%|
|10번|	39.39	|45.16%	|39%|	38%	|35%|	40%	|20%|